<div style="background-color: #8e7cc3; padding: 7px; text-align: center;">
    <h2>Projeto Machine Learning II | Algoritmos de SMV, Ensembles e K-means</h2>
</div>

## Sumário

1. Introdução
2. Importando Bibliotecas
3. Análise Exploratória dos Dados 
4. Pré-processamento
5. Modelos Supervisionados

    5.1. SMV

    5.2. Ensemble

6. Modelo Não-Supervisionado

    6.1. K-means

7. Conclusão

## 1. Introdução

##### **Objetivo:** 

Em universidades ao redor do mundo, a taxa de desistência de cursos superiores é um fator a ser proximamente analisado e combatido para garantir altos níveis de educação na população. Levantamos a hipótese de que diferentes variáveis sociais, econômicas e demográficas afetam a permanência ou desistência de estudantes de determinados cursos superiores. 

Para avaliar essa relação, criamos modelos de machine learning de aprendizado supervisionado para prever, com base nessas variáveis, se um estudante irá desistir ou graduar-se. 

Ainda, utilizamos modelos de aprendizado não supervisionado para identificar padrões e segmentações dentro dos dados dos estudantes, sem considerar previamente o desfecho de desistência ou graduação. Esses modelos permitem descobrir agrupamentos naturais e características comuns entre grupos de estudantes, fornecendo insights adicionais sobre os fatores que influenciam a desistência.

##### **Dados:** 

Utilizamos nesse projeto um conjunto de dados do Kaggle, encontrado [aqui](https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention). Esse foi criado a partir de pesquisa de uma instituição de ensino superior (adquirido de várias bases de dados distintas), relacionado a estudantes matriculados em diferentes cursos de graduação, como agronomia, design, educação, enfermagem, jornalismo, gestão, serviço social e tecnologias.


##### **Abaixo, a descrição das variáveis que utilizamos do dataset:** 

| Coluna                    | Descrição                                      | 
|---------------------------|------------------------------------------------|
| Marital status | Estado civil do aluno |
| Application mode | Método de inscrição utilizado pelo aluno |
| Application order |Ordem em que o aluno se inscreveu |
| Course   | Curso do aluno |  
| Daytime/evening attendance | Aulas diurnas ou noturnas | 
| Previous qualification | Qualificação do aluno antes de ingressar no ensino superior | 
| Nacionality | Nacionalidade do aluno |
| Mother's qualification | Qualificação da mãe do aluno | 
| Father's qualification | Qualificação do pai do aluno | 
| Mother's occupation | Ocupação da mãe do aluno |                         
| Father's occupation | Ocupação do pai do aluno | 
| Displaced | Se o aluno é uma pessoa deslocada | 
| Educational special needs | Se o aluno tem necessidades educacionais especiais | 
| Debtor | Se o aluno é devedor | 
| Tuition fees up to date | Se as mensalidades do aluno estão em dia | 
| Gender | Gênero do aluno | 
| Scholarship holder | Se o aluno é bolsista | 
| Age at enrollment | Idado do aluno na matrícula | 
| International | Se o aluno é estudante internacional | 
| Unemployment rate | Taxa de desemprego | 
| Inflation rate | Taxa de inflação | 
| GDP | Produto Interno Bruto | 
| Target | Se o aluno é "desistente" ou "graduado" | 

## 2. Importando Bibliotecas

In [22]:
# Manipulação e Visualização dos dados

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Pré-processamento

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Modelagem

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


# Métricas de Avaliação

from sklearn.metrics import accuracy_score

## 3. Análise Exploratória dos Dados

In [47]:
df = pd.read_csv(r'..\data\dataset.csv')
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [48]:
# removendo colunas que não serão utilizadas nas nossas análises e modelos

colunas_drop = [coluna for coluna in df.columns if 'Curricular units' in coluna]
print(colunas_drop)

['Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)']


In [49]:
df = df.drop(columns=colunas_drop)
df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,1,1,0,20,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,0,1,0,19,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,0,1,0,19,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,1,0,0,20,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,1,0,0,45,0,13.9,-0.3,0.79,Graduate


## 4. Pré-Processamento

In [2]:
raw_data = pd.read_csv(r'..\data\dataset.csv')

In [3]:
colunas_drop = [coluna for coluna in raw_data.columns if 'Curricular units' in coluna]
colunas_drop

['Curricular units 1st sem (credited)',
 'Curricular units 1st sem (enrolled)',
 'Curricular units 1st sem (evaluations)',
 'Curricular units 1st sem (approved)',
 'Curricular units 1st sem (grade)',
 'Curricular units 1st sem (without evaluations)',
 'Curricular units 2nd sem (credited)',
 'Curricular units 2nd sem (enrolled)',
 'Curricular units 2nd sem (evaluations)',
 'Curricular units 2nd sem (approved)',
 'Curricular units 2nd sem (grade)',
 'Curricular units 2nd sem (without evaluations)']

In [4]:
colunas_categoricas = ['Marital status', 'Application mode', 'Course', 'Daytime/evening attendance',
                        'Previous qualification', 'Nacionality', "Mother's qualification",
                        "Father's qualification", "Mother's occupation", "Father's occupation",
                        'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date',
                        'Gender', 'Scholarship holder', 'International']

colunas_numericas = ['Age at enrollment', 'Application order']

In [5]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()) 
])

In [6]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputing missing values with the most frequent category
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, colunas_numericas),
        ('cat', categorical_transformer, colunas_categoricas)
    ])


In [8]:
dados_refinados = raw_data.drop(columns=colunas_drop)

In [19]:
dados_refinados.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,1,1,0,20,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,0,1,0,19,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,0,1,0,19,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,1,0,0,20,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,1,0,0,45,0,13.9,-0.3,0.79,Graduate


In [9]:
X = dados_refinados.drop('Target', axis=1)
y = dados_refinados['Target']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train_preparado = preprocessor.fit_transform(X_train)
X_test_preparado = preprocessor.transform(X_test)

In [11]:
print(X_train_preparado.shape, X_test_preparado.shape)

(3539, 229) (885, 229)


## 5. Modelos Supervisionados

### 5.1 SVM

In [12]:
svm_parametros_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear']
}

svm_grid_search = GridSearchCV(SVC(random_state=0), svm_parametros_grid, cv=3, scoring='accuracy')
svm_grid_search.fit(X_train_preparado, y_train)

svm_params = svm_grid_search.best_params_
svm_score = svm_grid_search.best_score_

In [13]:
svm_model = SVC(C=svm_params['C'], kernel=svm_params['kernel'])

svm_model.fit(X_train_preparado, y_train)

svm_predictions = svm_model.predict(X_test_preparado)

svm_test_accuracy  = accuracy_score(y_test, svm_predictions)

print(f"Acurácia para a SVM: {svm_test_accuracy:.2f}")

Acurácia para a SVM: 0.65


### 5.2 Random Forest (Ensemble)

In [ ]:
rf_parametros_grid = {
    'n_estimators': [100,200,300],
    'max_features': ['auto', 'sqrt']
}

rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=0), rf_parametros_grid, cv=3, scoring='accuracy')
rf_grid_search.fit(X_train_preparado, y_train)

rf_params = rf_grid_search.best_params_
rf_score = rf_grid_search.best_score_

In [15]:
print(f"Melhores parâmetros para RF: {rf_params}")

Melhores parâmetros para RF: {'max_features': 'sqrt', 'n_estimators': 300}


In [16]:
rf_model = RandomForestClassifier(
    n_estimators=rf_params['n_estimators'],
    max_features=rf_params['max_features'],
    random_state=0
)

rf_model.fit(X_train_preparado, y_train)

rf_predictions = rf_model.predict(X_test_preparado)

rf_test_accuracy = accuracy_score(y_test, rf_predictions)

print(f"Acurácia para a RF: {rf_test_accuracy:.2f}")

Acurácia para a RF: 0.64


## 6. Modelo Não Supervisionado

### 6.1 K-means

## 7. Conclusão 